# Session 3: Advanced plotting


This is a bit of a catch-all final session, where we will have a look at:

- Heatmaps
- Some basic interactive plots (incl. some geospatial)

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# some extra specific imports - will be explained when they come up in the cells below

# this is a functionality from Matplotlib
from mpl_toolkits import axes_grid1

# PLotly Express library - the easy way to start with PLotly
import plotly.express as px

## Heatmaps

There are lots of different ways to build heatmaps, let's look at some of them and put that knowledge of colourmapping to the test.

### Heatmaps with image data

Heatmaps of images are useful across a range of applications:

- Remote sensing: DEMs
- Material sciences/geology/crystallography: BSE, EBSD
- Biology: cell segmentation and classification etc.

We're going to load in this [image of Mars](https://astrogeology.usgs.gov/search/map/mars_2020_terrain_relative_navigation_hirise_orthorectified_image_mosaic). At first, we're not going to worry about projection etc., and just treat it as an image.

For full geospatial analysis, use [Geopandas](https://geopandas.org/en/stable/) to prep your data, ensure you're using the correct projection system etc., and for procesing Shape files. It also contains some basic plotting functionality. I'd recommend [Basemap](https://matplotlib.org/basemap/stable/index.html) for static, publication-ready plots: this is developed by the Matplotlib team.
That's outside of the scope of this course: the steps are very similar to what's shown here, but require introduction of different file types/projections etc. that would be too much new information for people outside geospatial subjects!

In [ ]:
!wget -O mars.jpg https://astrogeology.usgs.gov/ckan/dataset/cee23e0f-a7fb-4695-b1c8-f295f09a305f/resource/2784f141-e40a-4291-a67b-0c458077b6ab/download/jez_hirise_soc_006_orthomosaic_25cm_eqc_latts0_lon0_first_crop1024.jpg

In [ ]:
mars = plt.imread("mars.jpg", format='jpeg')

In [ ]:
# we can very quickly preview what we've loaded:

plt.imshow(mars)

In [ ]:
# now let's do it properly with our object oriented code
# and add a colourbar

fig, ax = plt.subplots(layout="constrained")

im = ax.imshow(mars)

fig.colorbar(im)

In [ ]:
# How do we make the colourbar "fit" better?
# In the previous section, the colourbar was constrained by the grid layout
# but in this case we need to do some extra work
# we need to import the extra matplotlib tool "axes_grid1" which provides the
# function "axes_grid1.make_axes_locatable()"

fig, ax = plt.subplots(layout="constrained")

im = ax.imshow(mars)

divider = axes_grid1.make_axes_locatable(ax)

# create a colourbar axes: cax
cax = divider.append_axes("right", size="5%", pad=0.05)

# the fig.colorbar() function takes the argument cax
fig.colorbar(im, cax=cax)

Now we are going to fix the axes ticks and colourbar scale: in your work, you'll know what the limits should be. In this case, I'm taking a slightly handwaving approach to this dataset, based on the [information provided here](https://astrogeology.usgs.gov/search/map/mars_2020_terrain_relative_navigation_hirise_orthorectified_image_mosaic) and my imagination in terms of elevation, **purely for illustrative purposes**.

I'm going to scale the colourbar and the axes to fit the data.

In [ ]:
fig, ax = plt.subplots(layout="constrained")

lat_min = 18.3068
lat_max = 18.6693
lon_min = 77.2229
lon_max = 77.789

coord_extent = [lon_min, lon_max, lat_min, lat_max]

im = ax.imshow(mars,
               vmin=12, vmax=157, # vmax and vmin set the vertical extent (of the colourbar)
               extent=coord_extent) # this sets the x,y extent of the image

ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')

# colourbar code
divider = axes_grid1.make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, cax=cax, label='Elevation (m)')

You can also choose to hide the colourbar and axes ticks if you wish: for some fields, it's more typical to include a scalebar, and the  colourscale may be more relative than attached to a specific number (qualitative, like BSE images, EDS etc.)

In [ ]:
fig, ax = plt.subplots(layout="constrained")

lat_min = 18.3068
lat_max = 18.6693
lon_min = 77.2229
lon_max = 77.789

coord_extent = [lon_min, lon_max, lat_min, lat_max]

# keep the scaling if necessary for correct interpretation
im = ax.imshow(mars,
               vmin=12, vmax=157, # vmax and vmin set the vertical extent (of the colourbar)
               extent=coord_extent) # this sets the x,y extent of the image

# remove all of this:
# ax.set_xlabel('Longitude (°)')
# ax.set_ylabel('Latitude (°)')

# # colourbar code
# divider = axes_grid1.make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# fig.colorbar(im, cax=cax, label='Elevation (m)')

# turn off just the ticks, but keep the "spines" (bounding rectangle):
ax.set_xticks([])
ax.set_yticks([])

# or, turn off the spines, ticks, etc all in one go:
# ax.axis('off')

## Using Plotly

I have a bit of a love-hate relationship with the PLotly library!

- Easy interactive plots, lot's of different plot types
- Customisation is limited
- No inbuilt way to save a high-quality static plot:
    - There are external libraries to save out plots,
    - But I'd really recommend avoiding this for production of publication-quality static plots

But it's really good experience to apply your plotting knowledge to a new library!

Similarly to Matplotlib, it has a number of ways of interacting with it (although it's not really divided along "object oriented" vs. statebased):

- Plotly Express - high-level API, usually full figure functions (like figure-level functions in Seaborn)
- Plotly Graph Objects - more customisable, object-level access
- Plotly Dash (for building full web applications)

If you want to use this library extensively, I'd recommen working through their tutorials for using the graph objects, once you're comfortable with Plotly Express. Today, we'll use Plotly Express, because the underlying Graph Objects moe is a step up in complexity from what we've been doing so far!

We already imported it as `px`, so let's use it!

In [ ]:
# generate some quick random data

x = np.random.rand(50)
y = np.random.rand(50)
z = np.linspace(0, 1, 50)

In [ ]:
fig = px.scatter(x=x, y=y)
fig.show() # we need this for it to render

In [ ]:
# changing the x and y labels is a little bit different

fig = px.scatter(x=x, y=y,
                 labels={
                     "x":"X Value",
                     "y": "Y Value"
                 },
                 title="Title")
fig.show()

Let's load in our dataset from yesterday - the penguins, and plot some proper graphs.

In [ ]:
penguins = sns.load_dataset("penguins")

In [ ]:
penguins.columns

In [ ]:
fig = px.scatter(penguins,
                 x="bill_length_mm", y="bill_depth_mm",
                 labels={
                     "bill_length_mm":"Bill Length [mm]",
                     "bill_depth_mm": "Bill Depth [mm]"
                 },
                 title="Title")
fig.show()

In [ ]:
fig = px.scatter(penguins,
                 x="bill_length_mm", y="bill_depth_mm", color="species",
                 labels={
                     "bill_length_mm":"Bill Length [mm]",
                     "bill_depth_mm": "Bill Depth [mm]"
                 },
                 title="Title")
fig.show()

Changig the colour palette is also a little different, see this page of [the docs on discrete colours](https://plotly.com/python/discrete-color/#controlling-discrete-color-order)

In [ ]:
fig = px.scatter(penguins,
                 x="bill_length_mm", y="bill_depth_mm", color="species",
                 labels={
                     "bill_length_mm":"Bill Length [mm]",
                     "bill_depth_mm": "Bill Depth [mm]"
                 },
                 title="Title",
                 color_discrete_sequence=px.colors.qualitative.G10,)
fig.show()

In [ ]:
fig = px.scatter(penguins,
                 x="bill_length_mm", y="bill_depth_mm", color="species",
                 facet_col="island",
                 labels={
                     "bill_length_mm":"Bill Length [mm]",
                     "bill_depth_mm": "Bill Depth [mm]"
                 },
                 title="Title",
                 color_discrete_sequence=px.colors.qualitative.G10,)
fig.show()

In [ ]:
fig = px.scatter_matrix(penguins,
                        dimensions=["bill_length_mm", 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g',],
                        color="species")

# fig.update_traces(diagonal_visible=False)
# fig.update_traces(showupperhalf=False)
fig.show()

## Exercise

- Try to build a box-plot in plotly express
- View the [documentation on box plots](https://plotly.com/python/box-plots/#box-plot-with-plotlyexpress)
- Use the penguin data you have already loaded!